## MGMTMSA 403 Homework 2: Portfolio Optimization


## Pre-processing step: Estimate expected returns and covariance

In [76]:
# Import gurobi and numpy
from gurobipy import *
import numpy as np
from numpy import genfromtxt
import csv
import pandas as pd

## Get index of 4 tickers
tick4 = ["MSFT","GS","PG","SCHP"]

# Get variable names
with open('Prices.csv') as csvFile:
    reader = csv.reader(csvFile)
    tickers = next(reader) ## stores the tickets of all 390 stocks

tickind =[]
for t in tick4:
    tickind.append(tickers.index(t)) ## collect index that corresponds to each ticker

# Load data
prices = genfromtxt('Prices.csv', delimiter=',',skip_header = 1)

# get dimensions of data
d = prices.shape[0]
n = prices.shape[1]

# calculate monthly returns of each stock
returns = np.zeros((d-1,n))
for stock in range(n):
    for month in range(d-1):
        returns[month,stock] = prices[month+1,stock]/prices[month,stock]-1
        
# Store average return (parameter r_i in portfolio optimization model)       
avg_return = np.zeros(n)
avg_return = np.mean(returns,axis=0)

# Store covariance matrix (parameter C_ij in portfolio optimization model)
stocks_var_all = np.zeros((n,n))
stocks_var_all = np.cov(np.transpose(returns))


__Model 1.__ Start by focusing on a four-asset portfolio: Suppose you can only invest in Microsoft (MSFT), Goldman Sachs (GS), Proctor & Gamble (PG), and U.S. Treasury Bonds (SCHP). Con- struct a minimum-variance portfolio with an expected monthly return of at least 0.5%.

In [77]:
tickind

[315, 216, 372, 388]

In [78]:
stock_4 = avg_return[tickind]

In [79]:
# set model
mod = Model()
# Variable Identifiers
stocks = range(4)

# Decision variables
w = mod.addVars(len(stocks))

# Constraints
min_return = mod.addConstr(sum([w[i]*stock_4[i] for i in stocks]) >= .005)
probability = mod.addConstr(sum([w[i] for i in stocks]) == 1)

no_neg = {}
for i in stocks:
    no_neg[i] = mod.addConstr(w[i] >= 0)

# Objective
mod.setObjective(sum([w[i]*w[k]*stocks_var[i,k] for i in stocks for k in stocks]), GRB.MINIMIZE)

In [80]:
# Update and solve
mod.update()
mod.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6 rows, 4 columns and 12 nonzeros
Model fingerprint: 0xfd1dae80
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e-05, 7e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 4 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 8 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.87864836e+05 -1.87864836e+05  3.50e+03 1.91e-07  1.00e+06     0s
   1   5.03195868e+

In [81]:
print([f'{tick4[i]} {round(w[i].x,3)}' for i in stocks])
mod.objval

['MSFT 0.237', 'GS 0.026', 'PG 0.0', 'SCHP 0.737']


0.00017749326516578

### Write down the optimal risk (i.e. the optimal objective function value), solver time, and the weight on each of the four stocks.

The minimized variance was 0.0177%. This model was solved in .04 seconds. The weights were:
* MSFT: 0.237
* GS: 0.026
* PG: 0.0
* SCHP: 0.737

__Model 2.__ Now suppose you can invest in all 390 stocks. Construct a minimum-variance portfolio with an expected monthly return of at least 0.5%.

In [82]:
# set model
mod2 = Model()

# Variable Identifiers
stocks_all = range(n)

# Decision variables
w_all = mod2.addVars(n)

# Constraints
min_return2 = mod2.addConstr(sum([w_all[i]*avg_return[i] for i in stocks_all]) >= .005)
probability2 = mod2.addConstr(sum([w_all[i] for i in stocks_all]) == 1)

no_neg2 = {}
for i in stocks_all:
    no_neg[i] = mod2.addConstr(w_all[i] >= 0)

# Objective
mod2.setObjective(sum([w_all[i]*w_all[k]*stocks_var_all[i,k] for i in stocks_all for k in stocks_all]), GRB.MINIMIZE)

In [83]:
# Update and solve
mod2.update()
mod2.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 392 rows, 390 columns and 1170 nonzeros
Model fingerprint: 0x7b7d38a0
Model has 76245 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 390 columns, 780 nonzeros
Presolved model has 76245 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 59
 AA' NZ     : 1.830e+03
 Factor NZ  : 1.891e+03
 Factor Ops : 7.753e+04 (less than 1 second per iteration)
 Threads    : 4

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.89821559e-13 -2.89821559e-13  3.90e+05 3.22e-07  1.00e+06     0s


In [84]:
print([f'{tickers[i]} {round(w_all[i].x,4)}' for i in stocks_all if w_all[i].x >= .0001])
mod2.objval

['ABBV 0.0114', 'ABMD 0.0067', 'ATVI 0.0137', 'ANET 0.0118', 'AIZ 0.0222', 'ATO 0.0291', 'BBY 0.0026', 'CME 0.0128', 'ED 0.0002', 'DRI 0.0067', 'RE 0.0023', 'GWW 0.0078', 'HAS 0.0025', 'HCA 0.0296', 'HUM 0.0227', 'INFO 0.0517', 'ICE 0.0142', 'KEYS 0.0222', 'LLY 0.0085', 'LMT 0.0007', 'PSX 0.008', 'PNC 0.0323', 'BND 0.6805']


2.8785375543939184e-05

### Write down the optimal risk and solver time.

The minimized variance was 0.002878%. This model was solved in .07 seconds.

__Model 3.__ In practice, there are transaction fees associated with buying stocks. One way of keeping transaction fees low while still attaining desirable performance is to limit the total number of stocks that are purchased (i.e. limit the number of stocks that have a strictly positive weight). Construct a minimum-variance portfolio that selects at most 4 of the 390 stocks, and has an expected monthly return of at least 0.5%. (Note: By introducing binary variables into a quadratic program, we obtain a quadratic integer program. Fortunately, this particular __quadratic integer program__ can be solved by Gurobi.)

In [85]:
# set model
mod3 = Model()

# Decision variables
w_all4 = mod3.addVars(n)
w_bin = mod3.addVars(n, vtype = GRB.BINARY)

# Constraints
min_return3 = mod3.addConstr(sum([w_all4[i]*avg_return[i] for i in stocks_all]) >= .005)
probability3 = mod3.addConstr(sum([w_all4[i] for i in stocks_all]) == 1)
binary = mod3.addConstr(sum([w_bin[i] for i in stocks_all]) <= 4)

bin_op = {}
for i in stocks_all:
    bin_op[i] = mod3.addConstr(w_all4[i]*w_bin[i] == w_all4[i])

no_neg3 = {}
for i in stocks_all:
    no_neg[i] = mod3.addConstr(w_all4[i] >= 0)

# Objective
mod3.setObjective(sum([w_all4[i]*w_all4[k]*stocks_var_all[i,k] for i in stocks_all for k in stocks_all]), GRB.MINIMIZE)

In [86]:
# Update and solve
mod3.update()
mod3.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 393 rows, 780 columns and 1560 nonzeros
Model fingerprint: 0xc9d677c6
Model has 76245 quadratic objective terms
Model has 390 quadratic constraints
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Presolve time: 0.02s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Found heuristic solution: objective 0.0009032

Root relaxation: objective 2.878501e-05, 129 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexp

In [87]:
print([f'{tickers[i]} {round(w_all4[i].x,4)}' for i in stocks_all if w_all4[i].x > 0])
mod3.objval

['CME 0.1264', 'LLY 0.0755', 'NVDA 0.0438', 'BND 0.7544']


6.753470760728118e-05

### Report the optimal risk, solver time, and the ticker and weight on each of the four stocks selected by the model.

The minimized variance was 0.006753%. This model was solved in 57.23 seconds. The weights were:
* CME: 0.1264
* LLY: 0.0755
* NVDA: 0.0438
* BND: 0.7544

### Use your solution to Question 1 above to answer the following questions:

a) Is the optimal risk in Model 1 higher or lower than Model 2? Explain why in 1-2 sentences.  

The optimal risk is higher in Model 1. This is likely due to the fact that SCHP is the lowest variance investment but it does not return a high enough yeild to meet the .5% return constraint on its own. There is likely an investment in all 390 that has a higher return with a similarly low variance.

b) Is the optimal risk in Model 2 higher or lower than Model 3? Explain why in 1-2 sentences.

The optimal risk is lower in Model 2. Becuase were restricting our selection to 4 investments, we can not as efficiently minimize our variance while also meeting the investment return cosntraint. If the optimal number of selections was 4 or less, then the optimal risk would be the same.

### In some cases, we may want to get an approximate solution quickly by terminating the branch- and-bound algorithm before it finds an optimal solution. There are two ways to terminate Gurobi early: (a) by setting a maximum time limit, and (b) by setting a maximum acceptable optimality gap (the tolerance). Use Model 3 to answer the following two questions. For each part, also include your Gurobi output.

a) Set Gurobi to terminate after 30 seconds by including XYZ.Params.TimeLimit = 30.0 in your code for Model 3, where ’XYZ’ is the name of your model. How does the objective function value at termination compare the optimal value obtained in question 1c)?

In [88]:
mod3.Params.TimeLimit = 30.0 

Changed value of parameter TimeLimit to 30.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


In [90]:
# Update and solve
mod3.reset()
mod3.update()
mod3.optimize()

Discarded solution information
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 393 rows, 780 columns and 1560 nonzeros
Model fingerprint: 0xc9d677c6
Model has 76245 quadratic objective terms
Model has 390 quadratic constraints
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Presolve time: 0.01s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Found heuristic solution: objective 0.0009032

Root relaxation: objective 2.878501e-05, 129 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bou

In [91]:
mod3.objval

6.753470760728118e-05

The objective value is the same

b) Set Gurobi to terminate after reaching a gap of 10% by including XYZ.Params.MIPGap = 0.1 in your code for Model 3, where ’XYZ’ is the name of your model. (Note: The default gap in Gurobi is 0.0001.) How does the solver time compare with the solution time obtained in question 1c)?

In [94]:
mod3.Params.TimeLimit = 100.0 
mod3.Params.MIPGap = 0.1

Changed value of parameter TimeLimit to 100.0
   Prev: 30.0  Min: 0.0  Max: inf  Default: inf
Parameter MIPGap unchanged
   Value: 0.1  Min: 0.0  Max: inf  Default: 0.0001


In [95]:
# Update and solve
mod3.reset()
mod3.update()
mod3.optimize()

Discarded solution information
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 393 rows, 780 columns and 1560 nonzeros
Model fingerprint: 0xc9d677c6
Model has 76245 quadratic objective terms
Model has 390 quadratic constraints
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Presolve time: 0.01s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Found heuristic solution: objective 0.0009032

Root relaxation: objective 2.878501e-05, 129 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bou

In [ ]:
mod3.objval

The solver time is nearly 20 seconds faster when setting a looser gap restriction.